**Load data**

Load from csv into pandas. Each sequence, one by one.

In [1]:
import pandas as pd

from util.data import load_data, preprocess, make_dataset, split_train_test_validation

assets = load_data('data')

symbol = next(iter(assets.keys()))
asset = assets[symbol]
date_time = pd.to_datetime(asset.pop('date'), format='%Y-%m-%d')

plot_cols = ['close', 'volume']
plot_features = asset[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

asset.describe().transpose()

train, validation, test = split_train_test_validation(asset)

train = make_dataset(train, sequence_length=26, sequence_stride=1, shift=1)
validation = make_dataset(validation, sequence_length=26, sequence_stride=1, shift=1)
test = make_dataset(test, sequence_length=26, sequence_stride=1, shift=1)

print('train', train)
print('validation', validation)
print('test', test)

Tensor("strided_slice:0", shape=(None, 25, 6), dtype=float32) Tensor("stack:0", shape=(None, 0, 1), dtype=float32)
Tensor("strided_slice:0", shape=(None, 25, 6), dtype=float32) Tensor("stack:0", shape=(None, 0, 1), dtype=float32)
Tensor("strided_slice:0", shape=(None, 25, 6), dtype=float32) Tensor("stack:0", shape=(None, 0, 1), dtype=float32)
train <MapDataset shapes: ((None, 25, 6), (None, 0, 1)), types: (tf.float32, tf.float32)>
validation <MapDataset shapes: ((None, 25, 6), (None, 0, 1)), types: (tf.float32, tf.float32)>
test <MapDataset shapes: ((None, 25, 6), (None, 0, 1)), types: (tf.float32, tf.float32)>


**Strategies**
The ML models are evaluated using the following strategies:
* Average return
* Sharpe ratio

**Model**

Initialize Model

In [2]:
import tensorflow as tf
from tensorflow import keras

MAX_EPOCHS = 100

def compile_and_fit(model, train, val, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(train, epochs=MAX_EPOCHS,
                      validation_data=val,
                      callbacks=[early_stopping])
  return history

lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(units=1)
])

print(next(iter(train)))

print('Input shape:', next(iter(train))[0].shape)
print('Output shape:', lstm_model(next(iter(train))[0]).shape)


history = compile_and_fit(lstm_model, train, validation)

InvalidArgumentError: Incompatible shapes at component 1: expected [?,0,1] but got [32,26,1].

**Training**

Run training.